In [0]:
#mounting adls with databricks using SAS token
dbutils.fs.mount(
  source = "wasbs://sales@prajwalpanade.blob.core.windows.net/",
  mount_point = "/mnt/Salespoint",
  extra_configs = {"fs.azure.sas.sales.prajwalpanade.blob.core.windows.net": "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-30T16:38:56Z&st=2025-04-23T08:38:56Z&spr=https&sig=1TFdUI0AdSiQ2A%2FucT4%2BKMrvnOTH%2BftCqtNWN%2BvzFIo%3D"}
)

In [0]:
#fetching mounted point salespoint as named for mount point
dbutils.fs.ls("/mnt/Salespoint")

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/Salespoint/1000_Sales_Records.csv")
display(df)

In [0]:
%python
from pyspark.sql.functions import round

df = df.withColumn(
    "ProfitMargin", 
    round((df.TotalProfit / df.TotalRevenue) * 100, 2)    #rounded off with 2 decimal places
)
display(df.select("Country","ItemType","SalesChannel","OrderID","ProfitMargin"))

In [0]:
#toal revenue
total_rev = df.select("TotalRevenue").agg({"TotalRevenue": "sum"}).collect()[0][0]
print("Total Revenue: ", total_rev)

In [0]:
#Average Unit Price: The sum of TotalRevenue divided by the sum of UnitsSold. This gives you the average price at which items were sold.
avg_unit_price = df.select("TotalRevenue", "UnitsSold").agg({"TotalRevenue": "sum", "UnitsSold": "sum"})
print("Average Unit Price: ", avg_unit_price.collect()[0][0]/avg_unit_price.collect()[0][1])

In [0]:
print(df.columns)

In [0]:
#heighest revenue generating country
df.groupBy("Country").sum("TotalRevenue").orderBy("sum(TotalRevenue)", ascending=False).show()

In [0]:
%python
df_avg = df.groupBy("ItemType").avg("unitPrice").withColumnRenamed("avg(unitPrice)", "avg_unit_price")
df_avg.orderBy("avg_unit_price", ascending=False).show()

In [0]:
#heighest profit generating sales channel
df.groupBy("SalesChannel").sum("Totalprofit").orderBy("sum(Totalprofit)", ascending=False).show()

In [0]:
#What is the distribution of order priorities? Which priority level has the highest total revenue?
df.groupBy("OrderPriority").sum("TotalRevenue").orderBy("sum(TotalRevenue)", ascending=False).show()